# Introduction

This notebook is intended to provide a pointer towards the various calculations (and figures).
These are each in their own notebook (with shared code in modules). The following links provide
an overview (hopefully in a sensible order).

* [Calculation of phase equilibrium](./phase_equilibrium.ipynb)
* [Boundary layer thickness and simple falling time](./falling_timescale.ipynb)
* [Nucleation and growth rate calculation](./nucleation_and_growth.ipynb)
* [An equilibrium time regime plot](./regime_plot.ipynb)
* [Calculation of particle size to grow the core from snow](./particle_size.ipynb)
* [Calculation of particle size from Marsh's equations](./marsh_growth_size.ipynb)
* [Growth of a falling particle](./falling_particle.ipynb)

There are also two sets of notes on CSD calculations 
[here](./CSD_cooling.ipynb) and [here with added CD notes](./CSD_cooling_cdnotes.ipynb) as well as some left
over stuff below.


## OLD STUFF BELOW##

## Interface timescale. 

Here I implement equation 17 of Kirkpatrick (1974), "Kirkpatrick, R.J., 1975. Crystal growth from the melt: a review. American Mineralogist: Journal of Earth and Planetary Materials, 60(9-10), pp.798-814.". Below I refer to his equations as KX. 

- I assume that the growth rate is $da/dt$ where $a$ is particle radius
- Then time is $a/(da/dt)$
- I interpret this as the timescale required to grow the crystal to size a assuming that there are no diffusive hiderances to the process; the only obstacle seems to be the random detachment of particles from the surface according to K4.

- I take $f=1$
- Dario validates that K13 is a pretty good approximation; I have not included the correction in our eqn 2 here. 

The aassumption in K16 is checked in the cell kirk_growth17: it seems to be a bad approximation. Therefore we use eqn 14 in cell kirk_growth14. 

**However**

This model was found to be unsuitable because of the assumption that the rate constant can be described by a diffusion process (see eqn 9, which follows Turnbull and Cohen, 1960). It was replaced by the Sun et al study above. 

In [27]:
# Implement growth model of Kirkpatrick eqn 14, which does not invoke approximation K16
# Hc is latent heat
# Tl is liquidus
# eta is dynamic viscosity in Pa.s
def kirk_growth14(dT,Tl,eta,r):
    T = Tl - dT
    kB = 1.380649e-23 #J/K
    Rg = 8.314462
    f  = 1
    hf = 0.976e10
    Y  = (f*kB*T) / (3.0*np.pi*r**2*eta) * ( 1.0-np.exp( -(hf * dT)/(Rg*T*Tl) ) )
    return Y

In [28]:
# Implement growth model of Kirkpatrick eqn 17
# Hc is latent heat
# Tl is liquidus
# eta is dynamic viscosity in Pa.s
def kirk_growth17(dT,Tl,eta,r):
    kB = 1.380649e-23 #J/K
    Rg = 8.314462
    f  = 1
    hf = 0.976e10
    Y  = (f * kB) / (3.0 * np.pi * r**2 * eta) * (hf * dT) / (Rg * Tl)
    return Y

In [29]:
hf = 0.976e10 # From Dario
dT = 1.0      # 1 K undercooling
Tl = 6500.0   # Melting T of pure iron, high value
Tm_FeO = 5600.0
dT     = 200.0
tI     = np.zeros(len(r))

j = 0
for i in r:
    G = kirk_growth14(dT,Tm_FeO,0.01,i)
    tI[j] = i/G/secinyr
    j = j + 1

# 